In [29]:
import pinecone
from tqdm import tqdm
import pandas as pd

c:\Users\dattad\Anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [35]:

pinecone.init(api_key="", environment="")


In [36]:
pinecone.list_indexes()

['nytfb-dct-embeddings',
 'nytfb-gem-embeddings',
 'nytfb-glove-embeddings',
 'nytfb-laser-embeddings',
 'nytfb-openai-embeddings',
 'nytfb-random-embeddings',
 'nytfb-sentbert-embeddings']

In [5]:
import os
import pandas as pd
os.getcwd()

'c:\\Users\\dattad\\Desktop\\pinecone-poc'

In [6]:
data = pd.read_csv("data/NYTFB-280K/train.txt",sep = "\t",header=1,names = ["meta-1","meta-2","NE1","NE2","relation","text"])

C:\Users\dattad\AppData\Local\Temp\ipykernel_10644\919608044.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/NYTFB-280K/train.txt",sep = "\t",header=1,names = ["meta-1","meta-2","NE1","NE2","relation","text"])


In [7]:
dropped_nas = data.dropna()
dropped_nas["relation"]

0                    /location/location/contains
1                    /location/location/contains
2                    /location/location/contains
3                    /location/location/contains
4                    /location/location/contains
                           ...                  
228331    /location/neighborhood/neighborhood_of
228332    /location/neighborhood/neighborhood_of
228333    /location/neighborhood/neighborhood_of
228334    /location/neighborhood/neighborhood_of
228335               /location/location/contains
Name: relation, Length: 136962, dtype: object

In [10]:
len(dropped_nas["relation"].unique())

55

In [11]:
def clean_text(text):
    text = text.replace("-lrb-","").replace("-rrb-","").replace("###END###","")
    return text.strip()

In [12]:
dropped_nas['cleaned_text'] = dropped_nas['text'].apply(clean_text)

C:\Users\dattad\AppData\Local\Temp\ipykernel_10644\1914179460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped_nas['cleaned_text'] = dropped_nas['text'].apply(clean_text)


In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
import numpy as np
from collections import Counter

In [ ]:
def reduce_dataset(df,category,MIN_SAMPLE_PER_GROUP,TARGET_SAMPLES):

    small_groups = [group for _, group in df.groupby(category) if len(group) < MIN_SAMPLE_PER_GROUP]
    large_groups = [group for _, group in df.groupby(category) if len(group) >= MIN_SAMPLE_PER_GROUP]
    print(len(small_groups))
    print(len(large_groups))

    resampled_df = pd.concat(small_groups)
    remaining_samples = TARGET_SAMPLES - len(resampled_df)
    if remaining_samples > 0:
        samples_per_large_group = remaining_samples // len(large_groups)
        print(samples_per_large_group)
        for group in large_groups:
            print("--->",len(group))
            sampled_group = group.sample(n=samples_per_large_group, replace=False)
            resampled_df = pd.concat([resampled_df, sampled_group])

    resampled_df = resampled_df.reset_index(drop=True)
    return resampled_df

reduced_dataset = reduce_dataset(dropped_nas,"relation",400,10000)


In [ ]:
sentences = reduced_dataset["cleaned_text"].tolist()

embeddings= np.zeros((len(sentences),model.get_sentence_embedding_dimension()))

for i in tqdm(range(len(sentences)),desc = "Encoding Sentences"):
    embeddings[i] = model.encode([sentences[i]])

reduced_dataset["embedding"] = embeddings.tolist()


In [ ]:
reduced_dataset.to_csv("reduced_dataset.csv")

UPSERTING TO PINECONE

In [ ]:
# index_name = "pinecone-index"
# if index_name in pinecone.list_indexes():
#     pinecone.delete_index(index_name)

In [ ]:
reduced_dataset.reset_index(inplace=True)

In [ ]:
reduced_dataset.columns

Index(['index', 'meta-1', 'meta-2', 'NE1', 'NE2', 'relation', 'text',
       'cleaned_text', 'embedding'],
      dtype='object')

In [ ]:
def create_metadata(df):
    df["metadata"] = df.apply(lambda x: {"NE1":x["NE1"],"NE2":x["NE2"],"relation":x["relation"] },axis = 1)
    return df

In [ ]:
reduced_dataset = create_metadata(reduced_dataset)

CREATING PINECONE UPSERT VECTORS

In [ ]:
def create_vectors(df):
    df["pinecone-vectors"] = df.apply(lambda x: (str(x["index"]),x["embedding"],x["metadata"]),axis = 1)
    return df

reduced_dataset = create_vectors(reduced_dataset)

    

In [ ]:
upsert_vectors = reduced_dataset["pinecone-vectors"].to_list()

In [37]:
pinecone.list_indexes()

['nytfb-dct-embeddings',
 'nytfb-gem-embeddings',
 'nytfb-glove-embeddings',
 'nytfb-laser-embeddings',
 'nytfb-openai-embeddings',
 'nytfb-random-embeddings',
 'nytfb-sentbert-embeddings']

In [ ]:
# pinecone.create_index(index_name,dimension=384,metric="cosine")

In [41]:
index = pinecone.Index('nytfb-sentbert-embeddings')

UPSERT TO PINECONE

In [ ]:
#Upsert to the index with chunk size 500

vector_num = 500

for i in range(0,len(upsert_vectors),vector_num):
    chunk = upsert_vectors[i:i+vector_num]
    upsert_response = index.upsert(chunk)

USING PINECONE FOR FINDING SIMILARITY

query_response

In [ ]:
upsert_vectors[0][2]

{'NE1': 'wdet', 'NE2': 'detroit', 'relation': '/broadcast/content/location'}

In [ ]:
from collections import defaultdict

In [67]:
batch_size =100
top_k = 15
tuple_batches = [upsert_vectors[i:i+batch_size] for i in range(0,10000,batch_size)]

In [9]:
index_name = 'pinecone-index'
index = pinecone.Index(index_name)

In [18]:
results = index.query(top_k = 25,vector=upsert_vectors[0][1],include_metadata=True)



In [27]:
from collections import Counter

In [29]:
dict(Counter([result["metadata"]["relation"] for result in results["matches"]]).most_common())

{'/business/company/major_shareholders': 10,
 '/business/company_shareholder/major_shareholder_of': 4,
 '/business/company/founders': 4,
 '/broadcast/content/location': 2,
 '/people/person/place_of_birth': 2,
 '/business/company/place_founded': 2,
 '/people/person/place_lived': 1}

In [36]:
text_liste = [result["metadata"]["relation"] for result in results["matches"]]
text = "\n".join(text_liste)
       


In [34]:
text

'/broadcast/content/location\n/broadcast/content/location\n/people/person/place_of_birth\n/business/company/place_founded\n/business/company/place_founded\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/founders\n/people/person/place_lived\n/business/company/founders\n/business/company/founders\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/founders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/people/person/place_of_birth\n/business/company/major_shareholders\n/business/company/major_shareholders'

In [37]:
text2

'/broadcast/content/location\n/broadcast/content/location\n/people/person/place_of_birth\n/business/company/place_founded\n/business/company/place_founded\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/founders\n/people/person/place_lived\n/business/company/founders\n/business/company/founders\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/founders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/business/company_shareholder/major_shareholder_of\n/business/company/major_shareholders\n/business/company/major_shareholders\n/people/person/place_of_birth\n/business/company/major_shareholders\n/business/company/major_shareholders'

In [117]:

neighbours = defaultdict(set)
threshold = 0.55
index = pinecone.Index(index_name)

for batch in tqdm(tuple_batches):
    ids,vectors,_ = zip(*batch)
    results = index.query(top_k = 25,queries=vectors)
    for i,element in enumerate(results['results']):
        # if element["score"] > threshold:
        neighbours[str(ids[i])].update([res['id'] for res in element["matches"] if ((res['score'] > threshold) and (str(res['id']) != str(ids[i]) ))])


100%|██████████| 100/100 [06:40<00:00,  4.01s/it]


In [ ]:
len(neighbours)

10000

In [ ]:
neighbours_dict = {}
neighbours_dict = dict(neighbours)

In [ ]:
import pickle

with open('neigbours.pkl','wb') as f:
    pickle.dump(dict(neighbours),f)

with open('upsert_vectors.pkl','wb') as f:
    pickle.dump(upsert_vectors,f)

with open('reduced_dataset.pkl','wb') as f:
    pickle.dump(reduced_dataset,f)

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
import openai

import os

os.environ["OPENAI_API_TYPE"] = ""
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = ""
EMBEDDING_DEPLOYED_NAME = ""
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
openai.api_key = ""

def get_embeddings(text):
    embeddings =  OpenAIEmbeddings(
    deployment=EMBEDDING_DEPLOYED_NAME,
    model=EMBEDDING_MODEL_NAME)
    response = openai.Embedding.create(
            input=text,
            engine=EMBEDDING_DEPLOYED_NAME,
            deployment_id=EMBEDDING_DEPLOYED_NAME
        
)
    return response['data'][0]['embedding']
    # return embeddings.embed_query(text)



# SUMMARIZE NEIGBHOURHOOLD
THE VECTOR INDEX, EMBEDDING AND THE PINECONE INDEX ARE TO BE PASSED INTO THIS FUNCTION TO GET THE SUMMARY AND THE RELATIONS DICTIONARY

# SUMMARIZE NEIGBHOURHOOLD
THE VECTOR INDEX, EMBEDDING AND THE PINECONE INDEX ARE TO BE PASSED INTO THIS FUNCTION TO GET THE SUMMARY AND THE RELATIONS DICTIONARY

In [4]:
import pickle

with open('upsert_vectors.pkl','rb') as f:
    upsert_vectors = pickle.load(f)
with open('reduced_dataset.pkl','rb') as f:
    reduced_dataset = pickle.load(f)
with open('neigbours.pkl','rb') as f:
    neighbours = pickle.load(f)
with open('sentbert_50.pkl','rb') as f:
    consolidated_df = pickle.load(f)

In [5]:
consolidated_df.head(3)

,label,x,y,id,color,size,attributes.community,vector_index,sentence,relations,rel_ids,subjects,objects
0,193886,-2024.8311767578125,-1812.0592041015625,193886,"rgb(0,168,106)",10.0,14,193886,"When he joined the firm , UBS was a top-tier g...",['Missing'],['35'],['asia'],['ubs']
1,165213,4146.9443359375,-1740.330078125,165213,"rgb(219,115,0)",10.0,7,165213,"Oscar Brown Jr. , a singer , songwriter , play...",['Missing'],['35'],['oscar_brown_jr.'],['chicago']
2,93560,-3427.796142578125,-2922.0830078125,93560,"rgb(200,65,149)",10.0,32,93560,Other announcements about the New Sanctuary Mo...,['Missing'],['35'],['seattle'],['chicago']


In [83]:
def summarize_neighbourhood(_id,_embed,_index,_engine = 'Davinci-1',_relations = 'relations',_sentence = 'sentence',_k = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?'):
    
    if _index: 
        results = _index.query(top_k=_k + 1,vector = _embed,include_metadata=True)
        # since 1 of the results would be the input itself, I am searching for k+1 neighbours
        
        metadata_dict = dict(Counter([result["metadata"]["relation"] for result in results["matches"] if str(result["id"]) != str(_id) ]).most_common() )

        text_liste = [result["metadata"][_sentence] for result in results["matches"] if str(result["id"]) != str(_id)]
        
        text = "\n".join(text_liste)

        response = openai.Completion.create(
            engine= _engine,
            prompt=f'{prompt} \n"""\n{text}\n"""\n\n Theme:',
            temperature=0,
            max_tokens=_max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        summary = "Summary:" + response["choices"][0]["text"].replace("\n", "")
    else:
        summary = 'Pinecone index not found'
    
    return summary,metadata_dict


In [64]:
index

In [84]:
summarize_neighbourhood(_id = upsert_vectors[0][0],_embed = upsert_vectors[0][1],_index = index,_engine = 'Davinci-1',_relations = 'relation',_sentence = 'relation',_k = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?')

('Summary: The common theme in these texts is information related to companies, people, and locations.',
 {'/business/company/major_shareholders': 4,
  '/business/company/founders': 4,
  '/business/company/place_founded': 2,
  '/business/company_shareholder/major_shareholder_of': 2,
  '/broadcast/content/location': 1,
  '/people/person/place_of_birth': 1,
  '/people/person/place_lived': 1})

THE BELOW FUNCTION IS FOR SUMMARIZING CLUSTERS (SAMPLES_PER_CLUSTER = 15)

In [53]:
def dfs(_node, _cluster_id,_clusters,_neighbours):
    stack = [_node]
    while stack:
        _node = stack.pop()
        if _node not in _clusters:
            _clusters[_node] = _cluster_id
            stack.extend(_neighbours[_node] - set(_clusters.keys()))
    return _clusters

In [52]:
def run_clustering(_neighbours):
    clusters = {}
    cluster_id = 0

    for index in _neighbours.keys():
        if index not in clusters:
            clusters = dfs(index, cluster_id,clusters,_neighbours)
            cluster_id += 1

    return clusters

In [54]:
clusters = run_clustering(neighbours)

In [55]:
from collections import defaultdict
df = reduced_dataset.copy()
df["index"] = df["index"].astype(str)
df['cluster'] = df["index"].map(clusters)


In [56]:
umap_df = df.copy()

def all_vs_topklabelling(df,column,k):
    list_tuples = Counter(df[column]).most_common()[:k]
    top_k_labels = [item[0] for item in list_tuples]
    df["cluster_topk"] = df[column].apply(lambda x: x if x in top_k_labels else "-1")
    return df

umap_df = all_vs_topklabelling(umap_df,"cluster",10)

    


# SUMMARIZE CLUSTERS

In [24]:
import pandas as pd
def summarize_clusters(_dataframe,_text_column_name,_cluster_column_name,_cluster_labels,_engine = 'Davinci-1',_samples_per_cluster = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?'):
    
    output_dict = {"Cluster_Num" : [],"Summary" : []}

    for i in _cluster_labels:
        ctr_samples = _dataframe[_dataframe[_cluster_column_name] == i].shape[0]
        if ctr_samples < _samples_per_cluster:
            text = "\n".join(
                _dataframe[_dataframe[_cluster_column_name] == i][_text_column_name]
                .sample(ctr_samples, random_state = _random_state)
                .values
            )
        else:
            text = "\n".join(
                _dataframe[_dataframe[_cluster_column_name] == i][_text_column_name]
                .sample(_samples_per_cluster, random_state = _random_state)
                .values
            )

        response = openai.Completion.create(
            engine= _engine,
            prompt=f'{prompt} \n"""\n{text}\n"""\n\n Theme:',
            temperature=0,
            max_tokens=_max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        output_dict["Cluster_Num"].append(i)
        output_dict["Summary"].append(response["choices"][0]["text"].replace("\n", ""))
    return pd.DataFrame(output_dict)


In [9]:
consolidated_df.columns

Index(['label', 'x', 'y', 'id', 'color', 'size', 'attributes.community',
       'vector_index', 'sentence', 'relations', 'rel_ids', 'subjects',
       'objects'],
      dtype='object')

DATAFRAME EXPECTED FOR the SUMMARIZE_CLUSTERS FUNCTION

In [11]:
consolidated_df[['sentence','attributes.community']].head(4)

,sentence,attributes.community
0,"When he joined the firm , UBS was a top-tier g...",14
1,"Oscar Brown Jr. , a singer , songwriter , play...",7
2,Other announcements about the New Sanctuary Mo...,32
3,"On her first Manhattan weekend , her daughter ...",40


In [12]:
cluster_labels = consolidated_df["attributes.community"].unique().tolist()

In [21]:
from collections import Counter
Counter(consolidated_df["attributes.community"])

Counter({'14': 6063,
         '7': 4462,
         '32': 1251,
         '40': 10394,
         '22': 5654,
         '37': 2461,
         '8': 7790,
         '2': 11395,
         '24': 7252,
         '1': 13131,
         '15': 13704,
         '29': 4826,
         '16': 6016,
         '5': 2365,
         '26': 6799,
         '30': 1970,
         '35': 3187,
         '38': 5366,
         '34': 853,
         '12': 1840,
         '27': 5127,
         '4': 1595,
         '31': 752,
         '9': 9599,
         '20': 3909,
         '0': 6147,
         '33': 5582,
         '39': 465,
         '25': 3900,
         '10': 5260,
         '21': 2318,
         '23': 8,
         '6': 468,
         '3': 324,
         '36': 307,
         '17': 431,
         '19': 76,
         '11': 79,
         '41': 91,
         '18': 2,
         '13': 14,
         '28': 5})

In [25]:
cluster_summary = summarize_clusters(consolidated_df,"sentence","attributes.community",cluster_labels,_samples_per_cluster = 15,_random_state=17)

In [26]:
cluster_summary

,Cluster_Num,Summary
0,14,International Relations
1,7,"Death, Migration, and Change"
2,32,Religion
3,40,Accomplishments of notable people from variou...
4,22,Success and Failure The common theme in these...
5,37,Travel and Exploration
6,8,The common theme in these texts is that they ...
7,2,Politics and Government
8,24,Politics and Government
9,1,International Relations The common theme in t...


In [27]:
cluster_summary.to_csv("cluster_sentbert50_summary.csv")

In [53]:
def summarize_neighbourhood_without_embedding(_id,_index,_engine = 'Davinci-1',_relations = 'rel_idx',_sentence = 'text',_k = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?'):
    
    if _index: 
        results = _index.query(top_k=_k+1,id = _id,include_metadata=True)
        # print(results)
        # since 1 of the results would be the input itself, I am searching for k+1 neighbours
        
        metadata_dict = dict(Counter([result["metadata"][_relations][0] for result in results["matches"] if str(result["id"]) != str(_id) ]).most_common() )

        text_liste = [result["metadata"][_sentence] for result in results["matches"] if str(result["id"]) != str(_id)]
        
        text = "\n".join(text_liste)

        response = openai.Completion.create(
            engine= _engine,
            prompt=f'{prompt} \n"""\n{text}\n"""\n\n Theme:',
            temperature=0,
            max_tokens=_max_tokens,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
        )
        summary = "Summary:" + response["choices"][0]["text"].replace("\n", "")
    else:
        summary = 'Pinecone index not found'
    
    return summary,metadata_dict


In [55]:
index = pinecone.Index('nytfb-sentbert-embeddings')
summarize_neighbourhood_without_embedding(_id = "1",_index = index,_engine = 'Davinci-1',_relations = 'rel_idx',_sentence = 'text',_k = 15,_random_state = 42,_max_tokens = 100,prompt = 'What do the following texts have in common? Is there any common theme in these texts?')

('Summary: Growth and Development', {'35': 14, '48': 1})

A separate file to map 35 and 14 to the corresponding relations will also be uploaded later